In [ ]:
import numpy as np
import tensorflow as tf

In [2]:
with open('short_stories.txt', 'r', encoding='utf-8') as file:
    text = file.read()


In [3]:
stories = text.split('-END OF TEXT-')
stories = [story.strip() for story in stories if story.strip()] 

In [4]:
num_stories = len(stories)

print(f'Total number of stories: {num_stories}')

Total number of stories: 138


In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [6]:
total_words #number of unique words

3655

In [7]:
sequence_length = 15
# using the last 15 words to predict the next word

In [8]:
input_sequences = []
for line in text.split('-END OF TEXT-'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(sequence_length, len(token_list)):
        n_gram_sequence = token_list[i-sequence_length:i+1]
        input_sequences.append(n_gram_sequence)


In [9]:
max_sequence_length = sequence_length + 1  
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

In [10]:
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [11]:
X.shape

(13808, 15)

In [12]:
y.shape

(13808, 3655)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, BatchNormalization, Dropout

embedding_dim = 100
lstm_size = 128
# Define the model
model = Sequential()


model.add(Embedding(input_dim=total_words, output_dim=embedding_dim, input_length=max_sequence_length-1))
model.add(LSTM(lstm_size, return_sequences=True))
model.add(LSTM(lstm_size))
model.add(Dense(lstm_size, activation = "relu"))

model.add(Dense(total_words, activation='softmax'))


In [20]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 100)           365500    
                                                                 
 lstm_3 (LSTM)               (None, 15, 128)           117248    
                                                                 
 lstm_4 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 3655)              471495    
                                                                 
Total params: 1102339 (4.21 MB)
Trainable params: 1102339 (4.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
from tensorflow.keras.callbacks import EarlyStopping

In [23]:
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

model.fit(X, y, epochs=300, verbose=1, callbacks=[early_stopping])

Epoch 1/300
432/432 [==============================] - 13s 21ms/step - loss: 6.9043 - accuracy: 0.0545
Epoch 2/300
432/432 [==============================] - 9s 21ms/step - loss: 6.4734 - accuracy: 0.0548
Epoch 3/300
432/432 [==============================] - 10s 22ms/step - loss: 6.2971 - accuracy: 0.0579
Epoch 4/300
432/432 [==============================] - 11s 26ms/step - loss: 6.1252 - accuracy: 0.0641
Epoch 5/300
432/432 [==============================] - 11s 26ms/step - loss: 5.9724 - accuracy: 0.0671
Epoch 6/300
432/432 [==============================] - 11s 27ms/step - loss: 5.8362 - accuracy: 0.0720
Epoch 7/300
432/432 [==============================] - 14s 32ms/step - loss: 5.6837 - accuracy: 0.0833
Epoch 8/300
432/432 [==============================] - 13s 31ms/step - loss: 5.5288 - accuracy: 0.0932
Epoch 9/300
432/432 [==============================] - 14s 33ms/step - loss: 5.3803 - accuracy: 0.1028
Epoch 10/300
432/432 [==============================] - 13s 29ms/step - lo

In [24]:
seed_text = "Today is a beautiful day to "
next_words = 100

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)


1/1 [==============================] - 0s 16ms/step
Today is a beautiful day to  a hundred message it i was no one nevermore to return where you once seen someone my temporary hazel was decided to get the shot of sleep blocking hormones in order to stay awake all winter and guard the sleeping people i don't even mind the neurological damage caused by the sleep blocking hormone shot i don't even mind that i'm likely to die soon like almost all guards i'm just afraid for my species to follow me to the grave one of these winters next down and darkness you're hate said on mars writhing in pain on cots and


In [25]:
import gradio as gr


def generate_text(seed_text):
    next_words = 100
    generated_text = seed_text

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        seed_text += " " + output_word
        generated_text += " " + output_word

    return generated_text

iface = gr.Interface(fn=generate_text, inputs="text", outputs="text")
iface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [27]:
model.save("text_generation_model.keras")